# Caderno de testes e análises

## Criação de uma função de interpolação de uma curva de juros a partir dos preços dos títulos

In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
from scipy.optimize import *


### Criando a função de minimização pelo método não linear


In [26]:
def PrecoBondMatrizMin(xGuess):
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
    DFMat = np.power(DCMat,PzMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) - np.mat(b).T
    F = np.asarray(F).reshape(-1)
#     print(F,b)
    return F

######################################################################################################
##################################### funcao sem subtrair o preço do bond ############################
######################################################################################################

def PrecoBondMatriz(xGuess):
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
    DFMat = np.power(DCMat,PzMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) # - np.mat(b).T
    F = np.asarray(F).reshape(-1)
#     print(F,b)
    return F

######################################################################################################
##################################### funcao sem subtrair o preço do bond ############################
######################################################################################################

def CalculaFwdPzFixo(fwdsOriginais, vctosBonds, dtBase, vctosFixos):
    #Calcula o prazo original em DC
    fwd = [1] * len(listDtVctoFixo)
    for pz in listDtVctoFixo:
        for i in range(len(dtVctoBond)):
            if pz > (dtVctoBond[i] - dtBase).days:
                fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) ** (pz/360))
            else:
                if i == 0:
                    fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) ** (pz/360))
                    break
                else:
                    fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * (((1 + fwdsOriginais[i -1]) ** (((pz - (dtVctoBond[i-1]- dtBase).days) /360))))
                    break
    fwd = [x - 1 for x in fwd]
    return fwd

In [17]:
dir_input = "D:\\Documents\\JupyterNB\\tese_fgv\\dados\\dados_tratados\\"
MONTHS_PGTO_COUPON = [2, 5, 8, 11]
listDtVctoFixo = [30, 90, 360, 720]
VNA = [3154.28398]
arq_input = 'NTNB_2018.csv'
dfPrecos = pd.read_csv(dir_input + arq_input, delim_whitespace = True, index_col='Dia')

In [39]:
vcBonds = list(dfPrecos.columns)
outputFwd  = []
# TESTA SE É O ARQUIVO É DE NTNB (com fluxo de caixa) OU LTN (sem fluxo de caixa)
if ('LTN' in arq_input):
    freqCupom = 1
else:
    freqCupom = 2

i_aux = 0 # utilizado para fazer pontos de parada no for
# Inicio do calculo das fwds para cada um dos dias
for index, row in dfPrecos.iterrows():
    bonds = []
    b =[]
    for vcBond in vcBonds:
        
        # Cria o dicionário com os preços e vencimentos para cada dia
        bonds.append({'preco':row[vcBond], 'dtVcto':datetime.strptime(vcBond, '%d/%m/%Y').strftime('%Y-%m-%d'),
                           'freqCoupon': freqCupom})
        b.append(row[vcBond])    
    
    # Inicia a data base e VNA
    dtToday = datetime.strptime(index, "%Y-%m-%d")
    VNA = 3154.28398 # preciso criar uma rotina para compilar esses dados

    # Capturo algumas informações para o calculo das matrizes
    dtVctoBond = []
    for bond in bonds:
        dtVctoBond.append(datetime.strptime(bond['dtVcto'], '%Y-%m-%d'))

    maxVcto = max(dtVctoBond)
    dayVcto, monthVcto, yearVcto = maxVcto.day, maxVcto.month, maxVcto.year
    dayBase, monthBase, yearBase = dtToday.day, dtToday.month, dtToday.year

    # Calculo da matriz de cupons
    PzPgtoCupom = []
    mtzCoupon = []
    for year in range(dtToday.year, maxVcto.year + 1, 1):
        for month in MONTHS_PGTO_COUPON:
            if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
                PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
                vetorCoupon = []
                for bond in bonds:
                    bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
                                     + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
                                     for i in range(0, bond['freqCoupon'])]
                    if (month in bondPmtMonths) and datetime(year, month, 15) <= (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
                        vetorCoupon.append(1)
                    else:
                        vetorCoupon.append(0)
                mtzCoupon.append(vetorCoupon)
    mtzCoupon = np.transpose(mtzCoupon)
    
    # criando o vetor de cupons
    vlCupom = ((1 + 0.06)**(0.5) -1) * VNA
    arrayvlCupom = [vlCupom] * len(PzPgtoCupom)

    # criando a matriz de principais
    mtzPrincipal = []
    for k, bond in enumerate(bonds):
        mtzPrincipal.append(VNA * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))

    # Somando a matriz de cupons e de principais
    mtzVlCupom = (arrayvlCupom * mtzCoupon) + mtzPrincipal

    # Calculo das fwds para os vctos dos bonds
    xGuess = [0.1] * len(bonds)
    fwdsOriginais = fsolve(PrecoBondMatrizMin,xGuess)
#     print(fwdsOriginais)
    
    # Calculo das fwds para os prazos fixos
    fwdFixos = CalculaFwdPzFixo(fwdsOriginais, dtVctoBond, dtToday, listDtVctoFixo)
#     print(fwdFixos)
    outputFwd.append(fwdFixos)

#     if i_aux == 5:
#         break
    i_aux += 1


### Criar um arquivo de output em csv com data no index e prazos fixos no nome da coluna

In [42]:
dfOutputFwd = pd.DataFrame(outputFwd, index=dfPrecos.index, columns=listDtVctoFixo)
dfOutputFwd.to_csv(dir_input + "fwdReal_2018.csv")